In [4]:
import numpy as np 
import pandas as pd 

In [5]:
# this is the input data from Glide
dummy_data = 'kaas; pizza; ham, kaas; tonijn'

In [24]:
# preprocess data
data = dummy_data.split(';')

# create df
df = pd.DataFrame(data=[data], columns=['input1', 'input2', 'input3', 'input4'])

In [25]:
df

,input1,input2,input3,input4
0,kaas,pizza,"ham, kaas",tonijn
